<a href="https://colab.research.google.com/github/aleXyaO7/anti-Asian-BERT/blob/main/LabelPropagation_anti_asian_hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.9 MB/s 
     |████████████████████████████████| 101 kB 9.8 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
df = pd.read_excel('/content/drive/MyDrive/BERT/combined_v2.xlsx').sample(frac=1,random_state=0)
df.head()


,ID,Tweet ID,Text,label01,Source
24708,24709,1.304248e+18,"Good read and great reporting, as usual, from ...",0,AL
19760,19761,1.239712e+18,@eugenegu @realdonaldtrump #chinesevirus is tr...,0,Studies
27137,27138,3.133488e+09,So @realDonaldTrump sent tons of our PPE to Ch...,0,DH
2396,2397,1.214193e+18,@coffee_ted They are more like terrorists. #Po...,0,Studies
25904,25905,1.250981e+18,Quarantine got me thinking about being a veget...,0,AY


In [5]:
#convert label to integer
df = df.astype({"label01": int})
df.dtypes
df.head()

,ID,Tweet ID,Text,label01,Source
24708,24709,1.304248e+18,"Good read and great reporting, as usual, from ...",0,AL
19760,19761,1.239712e+18,@eugenegu @realdonaldtrump #chinesevirus is tr...,0,Studies
27137,27138,3.133488e+09,So @realDonaldTrump sent tons of our PPE to Ch...,0,DH
2396,2397,1.214193e+18,@coffee_ted They are more like terrorists. #Po...,0,Studies
25904,25905,1.250981e+18,Quarantine got me thinking about being a veget...,0,AY


In [6]:
df.shape

(27290, 5)

In [7]:
df['label01'].value_counts(normalize = True)

0    0.806303
1    0.193697
Name: label01, dtype: float64

In [8]:
df['Text'][0]

'Are we still allowed to quote ancient Chinese proverbs, or is that racist? #RacismIsAVirus'

In [9]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
tokenized = df['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [11]:
newt = [[tokenized[i][0], tokenized[i][1]] for i in range(len(tokenized))]

In [12]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
lp = LabelPropagation(kernel='knn')
lp.fit(newt, df['label01'])

In [13]:
from sklearn.semi_supervised import LabelPropagation
def labelprop(filename, outputfile):
  df2 = pd.read_csv(filename, header=0, names=['id', 'otherid', 'time', 'Text', 'xcord', 'ycord', 'device', 'place', 'city']).sample(frac=1,random_state=0)[['Text']]
  tokenized2 = df2['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  newt2 = []
  for i in tokenized2:
    newt2.append([i[0], i[1]])
  result = lp.predict(newt2)
  df2['label'] = result
  df2.to_csv(outputfile)

In [14]:
infile_header = '/content/drive/MyDrive/BERT/unlabeled/'
outfile_header = '/content/drive/MyDrive/BERT/labeled/'

In [16]:
for i in range(43):
  infile = infile_header + 'covid_tweets_unlabeled' + str(i) + '.csv'
  outfile = outfile_header + 'covid_tweets_labeled' + str(i) + '.csv'
  labelprop(infile, outfile)

Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors
